In [ ]:
import pandas as pd

# Load the dataset
file_path = "train.csv"  # Replace with your actual file path
data = pd.read_csv(file_path)

# Display the first few rows to verify the structure
print(data.head())


                                             Context  \
0  I'm going through some things with my feelings...   
1  I'm going through some things with my feelings...   
2  I'm going through some things with my feelings...   
3  I'm going through some things with my feelings...   
4  I'm going through some things with my feelings...   

                                            Response  
0  If everyone thinks you're worthless, then mayb...  
1  Hello, and thank you for your question and see...  
2  First thing I'd suggest is getting the sleep y...  
3  Therapy is essential for those that are feelin...  
4  I first want to let you know that you are not ...  


In [ ]:
import re

# Function to clean text
def clean_text(text):
    text = re.sub(r'\s+', ' ', str(text))  # Replace multiple spaces with single space
    text = re.sub(r'[^\w\s.,!?]', '', text)  # Remove special characters except punctuation
    return text.strip()

# Apply cleaning
data['Context_cleaned'] = data['Context'].apply(clean_text)
data['Response_cleaned'] = data['Response'].apply(clean_text)

# Check the cleaned data
print(data[['Context_cleaned', 'Response_cleaned']].head())


                                     Context_cleaned  \
0  Im going through some things with my feelings ...   
1  Im going through some things with my feelings ...   
2  Im going through some things with my feelings ...   
3  Im going through some things with my feelings ...   
4  Im going through some things with my feelings ...   

                                    Response_cleaned  
0  If everyone thinks youre worthless, then maybe...  
1  Hello, and thank you for your question and see...  
2  First thing Id suggest is getting the sleep yo...  
3  Therapy is essential for those that are feelin...  
4  I first want to let you know that you are not ...  


In [ ]:
from transformers import AutoTokenizer

# Load the tokenizer and set the padding token
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the cleaned context and response
data['Context_tokens'] = data['Context_cleaned'].apply(
    lambda x: tokenizer.encode(x, truncation=True, padding='max_length', max_length=128)
)
data['Response_tokens'] = data['Response_cleaned'].apply(
    lambda x: tokenizer.encode(x, truncation=True, padding='max_length', max_length=128)
)

# Check the tokenized data
print(data[['Context_tokens', 'Response_tokens']].head())


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

                                      Context_tokens  \
0  [3546, 1016, 832, 617, 1243, 351, 616, 7666, 2...   
1  [3546, 1016, 832, 617, 1243, 351, 616, 7666, 2...   
2  [3546, 1016, 832, 617, 1243, 351, 616, 7666, 2...   
3  [3546, 1016, 832, 617, 1243, 351, 616, 7666, 2...   
4  [3546, 1016, 832, 617, 1243, 351, 616, 7666, 2...   

                                     Response_tokens  
0  [1532, 2506, 6834, 345, 260, 28063, 11, 788, 3...  
1  [15496, 11, 290, 5875, 345, 329, 534, 1808, 29...  
2  [5962, 1517, 5121, 1950, 318, 1972, 262, 3993,...  
3  [35048, 12826, 318, 6393, 329, 883, 326, 389, ...  
4  [40, 717, 765, 284, 1309, 345, 760, 326, 345, ...  


In [ ]:
# Prepare the data as input-output pairs
train_data = [
    {
        "input_ids": context,
        "labels": response
    }
    for context, response in zip(data['Context_tokens'], data['Response_tokens'])
]


In [ ]:
import json

# Save the data to a JSON file
with open("tokenized_data.json", "w") as f:
    json.dump(train_data, f)

print("Tokenized data saved to tokenized_data.json")


Tokenized data saved to tokenized_data.json


In [ ]:
pip install transformers datasets torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import json

# Load the pre-trained DialoGPT model and tokenizer
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")

# Load the tokenized data
with open("tokenized_data.json", "r") as f:
    train_data = json.load(f)


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch

# Create a custom dataset
class ChatDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        return {
            "input_ids": torch.tensor(item["input_ids"], dtype=torch.long),
            "labels": torch.tensor(item["labels"], dtype=torch.long)
        }

# Initialize the dataset and DataLoader
dataset = ChatDataset(train_data)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)


In [ ]:
pip install transformers datasets torch


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the pre-trained DialoGPT model and tokenizer
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")

# Add a padding token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))  # Adjust model embeddings to match the tokenizer

def chat_with_bot():
    print("Chatbot is ready! Type 'exit' to end the chat.")

    # Initialize chat history (empty)
    chat_history_ids = None

    while True:
        try:
            # Get user input
            user_input = input("You: ")
            if user_input.lower() == 'exit':
                print("Goodbye!")
                break

            # Encode user input and append the conversation history (only the most recent input)
            input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors="pt")

            # Reset history and only keep the current input for context
            chat_history_ids = input_ids

            # Generate response based on current input
            chat_history_ids = model.generate(
                chat_history_ids,
                max_length=150,  # Increased max_length for more detailed responses
                pad_token_id=tokenizer.pad_token_id,
                temperature=0.75,  # Slightly higher temperature for more creative responses
                top_k=50,  # Keep top 50 tokens for diversity
                top_p=0.85,  # Lowering top_p for more focused sampling
                do_sample=True,  # Sampling enabled for more varied outputs
                num_return_sequences=1,  # Limit to one response at a time to avoid unnecessary outputs
                no_repeat_ngram_size=3  # Prevent repetition of the same n-grams
            )

            # Decode the response (only use the part after the current input)
            response = tokenizer.decode(chat_history_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)

            # Add logic to make the chatbot more serious about mental health
            if "dog" in user_input.lower() or "sad" in user_input.lower() or "heartbroken" in user_input.lower():
                response = "I'm really sorry you're feeling this way. It's important to talk to someone about your feelings. Have you thought about speaking to a therapist or counselor?"

            print(f"Chatbot: {response}")

        except Exception as e:
            print(f"An error occurred: {e}")
            break

# Start the chatbot
chat_with_bot()


Chatbot is ready! Type 'exit' to end the chat.
You: hi
Chatbot: Hey, how are you?
You: i fell sad lately
Chatbot: I'm really sorry you're feeling this way. It's important to talk to someone about your feelings. Have you thought about speaking to a therapist or counselor?
You: yes i have
Chatbot: Do you know what you're looking for?
You: i want an answer in my condition
Chatbot: I am not an expert, but I think the answer is in your condition.
You: can i tell about it
Chatbot: You can't
You: why i cant
Chatbot: What's your email?
You: wtf
Chatbot: I'm guessing it's an insult.
You: why you asking me an email
Chatbot: I'm not asking you an email. I'm asking you a question.
You: ok i dont know it early
Chatbot: I'll tell you in about 10 minutes.
You: what
Chatbot: Just a joke.
You: are you not serous
Chatbot: I'm not, but I'm not a child.
You: i need a mental health chat bot
Chatbot: You could always make one.
You: i make so do your job
Chatbot: I just want to be able to have fun in this ga

KeyboardInterrupt: Interrupted by user

In [ ]:
pip install transformers torch tqdm


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW  # Use PyTorch's AdamW optimizer
from transformers import AutoModelForCausalLM, AutoTokenizer, get_scheduler
from tqdm import tqdm
import json

# Load pre-trained model and tokenizer
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
tokenizer.pad_token = tokenizer.eos_token

# Load tokenized dataset
with open("tokenized_data.json", "r") as f:
    train_data = json.load(f)

# Create custom dataset class
class ChatDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        return {
            "input_ids": torch.tensor(item["input_ids"], dtype=torch.long),
            "labels": torch.tensor(item["labels"], dtype=torch.long),
        }

# Initialize dataset and dataloader
dataset = ChatDataset(train_data)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Set device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)  # Use PyTorch's AdamW
num_training_steps = len(train_loader) * 3  # Assume 3 epochs
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Training loop
model.train()
epochs = 3
progress_bar = tqdm(range(num_training_steps))

for epoch in range(epochs):
    for batch in train_loader:
        batch_input_ids = batch["input_ids"].to(device)
        batch_labels = batch["labels"].to(device)

        # Forward pass
        outputs = model(input_ids=batch_input_ids, labels=batch_labels)
        loss = outputs.loss

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        progress_bar.update(1)

    print(f"Epoch {epoch + 1}: Loss = {loss.item()}")

# Save the fine-tuned model
model.save_pretrained("fine_tuned_dialogpt")
tokenizer.save_pretrained("fine_tuned_dialogpt")

print("Model fine-tuned and saved to 'fine_tuned_dialogpt'")



  0%|          | 0/2634 [00:25<?, ?it/s]

 33%|███▎      | 879/2634 [06:33<13:03,  2.24it/s]

Epoch 1: Loss = 5.433384418487549



 67%|██████▋   | 1757/2634 [13:04<06:31,  2.24it/s]

Epoch 2: Loss = 5.665202617645264



100%|██████████| 2634/2634 [19:35<00:00,  2.25it/s]

Epoch 3: Loss = 5.058785915374756
Model fine-tuned and saved to 'fine_tuned_dialogpt'


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the pre-trained DialoGPT model and tokenizer
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")

# Add a padding token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))  # Adjust model embeddings to match the tokenizer

def chat_with_bot():
    print("Chatbot is ready! Type 'exit' to end the chat.")

    # Initialize chat history (empty)
    chat_history_ids = None

    while True:
        try:
            # Get user input
            user_input = input("You: ")
            if user_input.lower() == 'exit':
                print("Goodbye!")
                break

            # Encode user input and append the conversation history (only the most recent input)
            input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors="pt")

            # Reset history and only keep the current input for context
            chat_history_ids = input_ids

            # Generate response based on current input
            chat_history_ids = model.generate(
                chat_history_ids,
                max_length=150,  # Increased max_length for more detailed responses
                pad_token_id=tokenizer.pad_token_id,
                temperature=0.75,  # Slightly higher temperature for more creative responses
                top_k=50,  # Keep top 50 tokens for diversity
                top_p=0.85,  # Lowering top_p for more focused sampling
                do_sample=True,  # Sampling enabled for more varied outputs
                num_return_sequences=1,  # Limit to one response at a time to avoid unnecessary outputs
                no_repeat_ngram_size=3  # Prevent repetition of the same n-grams
            )

            # Decode the response (only use the part after the current input)
            response = tokenizer.decode(chat_history_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)

            # Add logic to make the chatbot more serious about mental health
            if "sad" in user_input.lower() or "depressed" in user_input.lower() or "heartbroken" in user_input.lower():
                response = "I'm really sorry you're feeling this way. It's important to talk to someone you trust or a mental health professional. Would you like some guidance on how to start this conversation?"

            # Provide serious responses when discussing mental health
            if "mental health" in user_input.lower():
                response = "Mental health is crucial, and it's okay to ask for help. You are not alone. Many people go through tough times, and reaching out for help is a strong step toward healing."

            print(f"Chatbot: {response}")

        except Exception as e:
            print(f"An error occurred: {e}")
            break

# Start the chatbot
chat_with_bot()


Chatbot is ready! Type 'exit' to end the chat.
You: hi
Chatbot: Hello!
You: what are you
Chatbot: A guy who got to live in a castle and was also a huge fan of Star Wars.


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-28-9677dde2570f>", line 64, in <cell line: 64>
    chat_with_bot()
  File "<ipython-input-28-9677dde2570f>", line 22, in chat_with_bot
    user_input = input("You: ")
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelbase.py", line 851, in raw_input
    return self._input_request(str(prompt),
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelbase.py", line 895, in _input_request
    raise KeyboardInterrupt("Interrupted by user") from None
KeyboardInterrupt: Interrupted by user

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'Ke

TypeError: object of type 'NoneType' has no len()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import pandas as pd

# Load the pre-trained DialoGPT model and tokenizer
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")

# Add a padding token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))  # Adjust model embeddings to match the tokenizer

# Load CSV data
def load_csv_data(file_path):
    try:
        data = pd.read_csv(file_path)
        return data
    except Exception as e:
        print(f"Error loading CSV: {e}")
        return None

# Retrieve an answer from the CSV data based on a user query
def get_csv_response(data, query):
    if data is not None and "Question" in data.columns and "Answer" in data.columns:
        for _, row in data.iterrows():
            if query.lower() in row["Question"].lower():
                return row["Answer"]
    return None

# Chatbot function with CSV and GPT integration
def chat_with_bot(csv_file_path):
    print("Chatbot is ready! Type 'exit' to end the chat.")
    chat_history_ids = None  # Initialize chat history
    csv_data = load_csv_data(csv_file_path)
    max_history_tokens = 300  # Limit conversation history tokens

    while True:
        try:
            # Get user input
            user_input = input("You: ").strip()
            if user_input.lower() == 'exit':
                print("Goodbye!")
                break

            # Check if the CSV contains a related response
            csv_response = get_csv_response(csv_data, user_input)
            if csv_response:
                print(f"Chatbot (from CSV): {csv_response}")
                continue

            # Encode user input and append to conversation history
            input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors="pt")
            if chat_history_ids is not None:
                chat_history_ids = torch.cat([chat_history_ids, input_ids], dim=-1)

                # Truncate history to the last `max_history_tokens`
                if chat_history_ids.shape[-1] > max_history_tokens:
                    chat_history_ids = chat_history_ids[:, -max_history_tokens:]
            else:
                chat_history_ids = input_ids

            # Generate response with adjusted parameters
            chat_history_ids = model.generate(
                chat_history_ids,
                max_length=200,
                pad_token_id=tokenizer.pad_token_id,
                temperature=0.7,
                top_k=40,
                top_p=0.8,
                do_sample=True,
                num_return_sequences=1,
                no_repeat_ngram_size=3
            )

            # Decode and sanitize the response
            response = tokenizer.decode(chat_history_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True).strip()
            if response:
                print(f"Chatbot: {response}")
            else:
                print("Chatbot: I'm not sure how to respond to that.")

        except KeyboardInterrupt:
            print("\nChatbot: Goodbye!")
            break

        except Exception as e:
            print(f"An error occurred: {e}")
            break

# Provide the path to your CSV file here
csv_file_path = "data.csv"  # Replace with your CSV file's path
chat_with_bot(csv_file_path)




Chatbot is ready! Type 'exit' to end the chat.
You: hi
Chatbot: Hi!
You: can you help me
Chatbot: can you help meI can, but I don't know how to help you.
You: about my feeling
Chatbot: can you help meI can, but I don't know how to help you.about my feelingAbout my feeling about my feeling?
You: yes
Chatbot: Hi!can you help meI can, but I don't know how to help you.about my feelingAbout my feeling about my feeling?yesI'm a girl, and I'm a boy.
You: i'm a boy
Chatbot: can you help meI can, but I don't know how to help you.about my feelingAbout my feeling about my feeling?yesI'm a girl, and I'm a boy.i'm a boyI am a girl.
You: I guess not. All I can think about are my exams.; not really; i guess not
Chatbot: help you.about my feelingAbout my feeling about my feeling?yesI'm a girl, and I'm a boy.i'm a boyI am a girl.I guess not. All I can think about are my exams.; not really; i guess notI feel you. I don t have a lot of confidence in myself.
You: I want some advice.; I need some advice.; 

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import pandas as pd

# Load the pre-trained DialoGPT model and tokenizer
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")

# Add a padding token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))  # Adjust model embeddings to match the tokenizer

# Load CSV data
def load_csv_data(file_path):
    try:
        data = pd.read_csv(file_path)
        return data
    except Exception as e:
        print(f"Error loading CSV: {e}")
        return None

# Retrieve an answer from the CSV data based on a user query
def get_csv_response(data, query):
    if data is not None and "Context" in data.columns and "Response" in data.columns:
        for _, row in data.iterrows():
            # Lowercase for case-insensitive matching
            if query.lower() in row["Context"].lower():
                return row["Response"]
    return None

# Chatbot function with CSV and GPT integration
def chat_with_bot(csv_file_path):
    print("Chatbot is ready! Type 'exit' to end the chat.")
    chat_history_ids = None  # Initialize chat history
    csv_data = load_csv_data(csv_file_path)
    max_history_tokens = 300  # Limit conversation history tokens

    while True:
        try:
            # Get user input
            user_input = input("You: ").strip()
            if user_input.lower() == 'exit':
                print("Goodbye!")
                break

            # Check if the CSV contains a related response
            csv_response = get_csv_response(csv_data, user_input)
            if csv_response:
                print(f"Chatbot (from CSV): {csv_response}")
                continue

            # Encode user input and append to conversation history
            input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors="pt")
            if chat_history_ids is not None:
                chat_history_ids = torch.cat([chat_history_ids, input_ids], dim=-1)

                # Truncate history to the last `max_history_tokens`
                if chat_history_ids.shape[-1] > max_history_tokens:
                    chat_history_ids = chat_history_ids[:, -max_history_tokens:]
            else:
                chat_history_ids = input_ids

            # Generate response with adjusted parameters
            chat_history_ids = model.generate(
                chat_history_ids,
                max_length=200,
                pad_token_id=tokenizer.pad_token_id,
                temperature=0.7,
                top_k=40,
                top_p=0.8,
                do_sample=True,
                num_return_sequences=1,
                no_repeat_ngram_size=3
            )

            # Decode and sanitize the response
            response = tokenizer.decode(chat_history_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True).strip()
            if response:
                print(f"Chatbot: {response}")
            else:
                print("Chatbot: I'm not sure how to respond to that.")

        except KeyboardInterrupt:
            print("\nChatbot: Goodbye!")
            break

        except Exception as e:
            print(f"An error occurred: {e}")
            break

# Provide the path to your CSV file here
csv_file_path = "data.csv"  # Replace with your CSV file's path
chat_with_bot(csv_file_path)

Chatbot is ready! Type 'exit' to end the chat.
You: I don't have any friends
Chatbot: I don t have any
You: Can I ask you something?
Chatbot: I don t have anyCan I ask you something?I think you have a better chance of finding a friend in a crowd than you do in a bar.

Chatbot: Goodbye!


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import pandas as pd

# Load the pre-trained DialoGPT model and tokenizer
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")

# Add a padding token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

# Load CSV data
def load_csv_data(file_path):
    try:
        data = pd.read_csv(file_path)
        return data
    except Exception as e:
        print(f"Error loading CSV: {e}")
        return None

# Retrieve an answer from the CSV data based on a user query
def get_csv_response(data, query):
    if data is not None and "Context" in data.columns and "Response" in data.columns:
        for _, row in data.iterrows():
            # Lowercase for case-insensitive matching
            if query.lower() in row["Context"].lower():
                return row["Response"]
    return None

# Chatbot function with CSV and GPT integration (modified)
def chat_with_bot(csv_file_path):
    print("Chatbot is ready! Type 'exit' to end the chat.")
    chat_history_ids = None
    csv_data = load_csv_data(csv_file_path)

    while True:
        try:
            user_input = input("You: ").strip()
            if user_input.lower() == 'exit':
                print("Goodbye!")
                break

            # Check if the CSV contains a related response
            csv_response = get_csv_response(csv_data, user_input)
            if csv_response:
                print(f"Chatbot (from CSV): {csv_response}")
                continue

            # Encode the new user input
            new_user_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

            # Prepare the bot's input (previous response + new user input)
            bot_input_ids = new_user_input_ids
            if chat_history_ids is not None:
                bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1)

            # Generate response with adjusted parameters
            chat_history_ids = model.generate(
                bot_input_ids,
                max_length=150,  # Reduced max_length for faster response
                pad_token_id=tokenizer.pad_token_id,
                temperature=0.6,  # Lowered temperature for faster/more focused response
                top_k=30,         # Reduced top_k for faster/more focused response
                top_p=0.8,
                do_sample=True,
                num_return_sequences=1,
                no_repeat_ngram_size=3
            )

            # Decode the response (only the new part generated by the bot)
            response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

            # Print the response
            print(f"Chatbot: {response}")

        except KeyboardInterrupt:
            print("\nChatbot: Goodbye!")
            break
        except Exception as e:
            print(f"An error occurred: {e}")
            break

# Provide the path to your CSV file here
csv_file_path = "data.csv"  # Replace with your actual CSV file path
chat_with_bot(csv_file_path)

Chatbot is ready! Type 'exit' to end the chat.
You: I want some advice
Chatbot: You're not going to get any help here.
You: why
Chatbot: Because you're not a good person.
You: oh im s good person you have not prove it
Chatbot: I'm not a bad person. I'm not.
You: exit
Goodbye!


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load fine-tuned model and tokenizer
model_path = "fine_tuned_dialogpt"  # Replace with your fine-tuned model path
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to generate a chatbot response
def generate_response(prompt, max_length=100):
    # Include a system prompt for context
    system_prompt = (
        "You are a helpful and empathetic therapist chatbot. Respond thoughtfully and constructively."
    )
    full_prompt = f"{system_prompt}\nUser: {prompt}\nChatbot:"

    # Tokenize the input with padding and attention mask
    inputs = tokenizer(
        full_prompt,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=128
    )
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    # Generate the response
    try:
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,         # Enable sampling
            temperature=0.6,        # Add moderate randomness
            top_k=100,              # Consider top 100 tokens
            repetition_penalty=1.3  # Penalize repetition more strongly
        )
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Extract chatbot's response after "Chatbot:"
        chatbot_response = response.split("Chatbot:", 1)[-1].strip()
        return chatbot_response
    except Exception as e:
        return f"Error generating response: {e}"

# Chat loop
def chatbot():
    print("Chatbot is ready! Type 'exit' to end the conversation.")
    while True:
        try:
            user_input = input("You: ").strip()
            if user_input.lower() == "exit":
                print("Chatbot: Goodbye!")
                break
            if not user_input:
                print("Chatbot: Please type something.")
                continue
            response = generate_response(user_input)
            print(f"Chatbot: {response}")
        except KeyboardInterrupt:
            print("\nChatbot: Goodbye!")
            break
        except Exception as e:
            print(f"Chatbot: An error occurred: {e}")

# Main function to run the chatbot
if __name__ == "__main__":
    chatbot()


Chatbot is ready! Type 'exit' to end the conversation.
You: hi
Chatbot: you to about of very you in feel like. we. I. as that the how this them would you and,
You: what
Chatbot: to you. your, will that the of It

Chatbot: Goodbye!


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import pandas as pd

# Load the pre-trained DialoGPT model and tokenizer
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")

# Add a padding token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))  # Adjust model embeddings to match the tokenizer

# Load CSV data
def load_csv_data(file_path):
    try:
        data = pd.read_csv(file_path)
        return data
    except Exception as e:
        print(f"Error loading CSV: {e}")
        return None

# Retrieve an answer from the CSV data based on a user query
def get_csv_response(data, query):
    if data is not None and "Question" in data.columns and "Answer" in data.columns:
        # Make sure to sanitize the query before matching
        query = query.lower().strip()
        for _, row in data.iterrows():
            if query in row["Question"].lower():
                return row["Answer"]
    return None

# Adjusted chatbot function with improved responses for mental health
def chat_with_bot(csv_file_path):
    print("Chatbot is ready! Type 'exit' to end the chat.")
    chat_history = []  # Maintain chat history as a list of strings
    csv_data = load_csv_data(csv_file_path)
    max_history_lines = 10  # Limit the conversation history to the last 10 exchanges for context

    previous_response = ""  # Keep track of the previous response to avoid repetition

    while True:
        try:
            # Get user input
            user_input = input("You: ").strip()
            if user_input.lower() == 'exit':
                print("Goodbye!")
                break

            # Handle empty input (no response from the user)
            if not user_input:
                print("Chatbot: Please enter a message.")
                continue

            # Check if the CSV contains a related response
            csv_response = get_csv_response(csv_data, user_input)
            if csv_response:
                print(f"Chatbot (from CSV): {csv_response}")
                previous_response = csv_response  # Store CSV response to avoid repeating
                continue

            # Check for common mental health phrases
            if any(phrase in user_input.lower() for phrase in ["sad", "depressed", "feeling down", "help", "lonely"]):
                response = "I'm really sorry you're feeling this way. It's important to talk to someone who can offer support, like a friend, family member, or a professional. You're not alone. Please reach out to someone."
            else:
                # Append user input to the history
                chat_history.append(f"You: {user_input}")
                if len(chat_history) > max_history_lines:
                    chat_history = chat_history[-max_history_lines:]  # Keep recent exchanges only

                # Prepare input for the model
                input_text = " ".join(chat_history)
                input_ids = tokenizer.encode(input_text + tokenizer.eos_token, return_tensors="pt")

                # Generate response with adjusted parameters for more controlled and empathetic responses
                output_ids = model.generate(
                    input_ids,
                    max_length=200,
                    pad_token_id=tokenizer.pad_token_id,
                    temperature=0.6,  # Lower temperature for more controlled responses
                    top_k=40,
                    top_p=0.85,  # Slightly increased top_p to allow a bit more variety while maintaining focus
                    do_sample=True,
                    num_return_sequences=1,
                    no_repeat_ngram_size=3
                )

                # Decode the response
                response = tokenizer.decode(output_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True).strip()

            # Ensure the response is meaningful and not repeating the last one
            if response.lower() == previous_response.lower():
                response = "I have already said that, let's talk about something else."

            # Add response to chat history and display it
            chat_history.append(f"Chatbot: {response}")
            print(f"Chatbot: {response}")

            # Store the response for the next round to avoid repetition
            previous_response = response

        except KeyboardInterrupt:
            print("\nChatbot: Goodbye!")
            break

        except Exception as e:
            print(f"An error occurred: {e}")
            break

# Provide the path to your CSV file here
csv_file_path = "datacsv"  # Replace with your CSV file's path
chat_with_bot(csv_file_path)


Chatbot is ready! Type 'exit' to end the chat.
You: hi
Chatbot: I'm not you, but I'm sure you've heard of me.
You: huh
Chatbot: Chatbot : Hi!
You: how are you?
Chatbot: I'm not a bot, I'm a human!
You: what
Chatbot: This bot is too good.
You: can you help me
Chatbot: I'm really sorry you're feeling this way. It's important to talk to someone who can offer support, like a friend, family member, or a professional. You're not alone. Please reach out to someone.
You: f
Chatbot: Chatbot : I'm still not sure if I should upvote
You: df
Chatbot: I'm still trying to figure out what that means.
You: what i stress?
Chatbot: I don't know what's
You: can you give me story
Chatbot: You : I want to talk about the weather.
You: go a head 
Chatbot: I want to give
You: what?
Chatbot: You : you can't give me what I want!
You: okay
Chatbot: I think the one with the best story was u theshoebox
You: okay
Chatbot: I think you mean u thesnoebox
You: yes
Chatbot: You : I don't know what I'm doing. Chatbot : I'

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import pandas as pd

# Load the pre-trained DialoGPT model and tokenizer
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")

# Add a padding token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))  # Adjust model embeddings to match the tokenizer

# Load CSV data
def load_csv_data(file_path):
    try:
        data = pd.read_csv(file_path)
        return data
    except Exception as e:
        print(f"Error loading CSV: {e}")
        return None

# Retrieve an answer from the CSV data based on a user query
def get_csv_response(data, query):
    if data is not None and "Context" in data.columns and "Response" in data.columns:
        # Make sure to sanitize the query before matching
        query = query.lower().strip()
        # Find the most relevant response
        for _, row in data.iterrows():
            # If query is part of the context (simple matching)
            if query in row["Context"].lower():
                return row["Response"]
    return None

# Chatbot function with CSV and GPT integration
def chat_with_bot(csv_file_path):
    print("Chatbot is ready! Type 'exit' to end the chat.")
    chat_history = []  # Maintain chat history as a list of strings
    csv_data = load_csv_data(csv_file_path)
    max_history_lines = 10  # Limit the conversation history to the last 10 exchanges for context

    while True:
        try:
            # Get user input
            user_input = input("You: ").strip()
            if user_input.lower() == 'exit':
                print("Goodbye!")
                break

            # Check if the CSV contains a related response
            csv_response = get_csv_response(csv_data, user_input)
            if csv_response:
                print(f"Chatbot (from CSV): {csv_response}")
                continue

            # Append user input to the history
            chat_history.append(f"You: {user_input}")
            if len(chat_history) > max_history_lines:
                chat_history = chat_history[-max_history_lines:]  # Keep recent exchanges only

            # Prepare input for the model
            input_text = " ".join(chat_history)
            input_ids = tokenizer.encode(input_text + tokenizer.eos_token, return_tensors="pt")

            # Generate response
            output_ids = model.generate(
                input_ids,
                max_length=200,
                pad_token_id=tokenizer.pad_token_id,
                temperature=0.7,
                top_k=40,
                top_p=0.8,
                do_sample=True,
                num_return_sequences=1,
                no_repeat_ngram_size=3
            )

            # Decode the response
            response = tokenizer.decode(output_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True).strip()

            # Add response to chat history and display it
            chat_history.append(f"Chatbot: {response}")
            print(f"Chatbot: {response}")

        except KeyboardInterrupt:
            print("\nChatbot: Goodbye!")
            break

        except Exception as e:
            print(f"An error occurred: {e}")
            break

# Provide the path to your CSV file here
csv_file_path = "train.csv"  # Replace with your CSV file's path
chat_with_bot(csv_file_path)


KeyboardInterrupt: 

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import pandas as pd

# Load the pre-trained GPT model and tokenizer
model = AutoModelForCausalLM.from_pretrained("fine_tuned_dialogpt")
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")

# Add padding token if missing
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

# Load CSV data for context-based responses
def load_csv_data(file_path):
    try:
        data = pd.read_csv(file_path)
        return data
    except Exception as e:
        print(f"Error loading CSV: {e}")
        return None

# Retrieve an answer from the CSV data based on a user query
def get_csv_response(data, query):
    if data is not None and "Context" in data.columns and "Response" in data.columns:
        # Lowercase the query and context for case-insensitive matching
        query = query.lower().strip()
        for _, row in data.iterrows():
            # Match if the query is a part of the context
            if query in row["Context"].lower():
                return row["Response"]
    return None

# Tone adjustment function (therapy-like language)
def adjust_tone(response):
    # Adding a therapeutic tone to the response
    therapeutic_responses = [
        "I'm really sorry you're feeling this way. It's okay to feel down sometimes.",
        "You're not alone in this, and it’s important to take one step at a time.",
        "It’s okay to not have everything figured out. Be kind to yourself.",
        "It sounds like you’re dealing with a lot. Remember, it’s okay to ask for help.",
        "I understand how difficult that must be. You're doing the best you can right now."
    ]
    return f"{therapeutic_responses[0]} {response}"  # Choose an appropriate therapeutic start for empathy

# Chatbot function that integrates CSV-based responses and the pre-trained model
def chat_with_bot(csv_file_path):
    print("Chatbot is ready! Type 'exit' to end the chat.")
    chat_history = []  # Maintain chat history
    csv_data = load_csv_data(csv_file_path)
    max_history_lines = 10  # Limit conversation history to recent exchanges

    while True:
        try:
            # Get user input
            user_input = input("You: ").strip()
            if user_input.lower() == 'exit':
                print("Goodbye!")
                break

            # Try getting a response from the CSV file
            csv_response = get_csv_response(csv_data, user_input)
            if csv_response:
                print(f"Chatbot (from CSV): {adjust_tone(csv_response)}")
                continue  # Proceed to the next round of conversation

            # Append user input to the chat history
            chat_history.append(f"You: {user_input}")
            if len(chat_history) > max_history_lines:
                chat_history = chat_history[-max_history_lines:]  # Keep recent exchanges only

            # Prepare the conversation history for the model
            input_text = " ".join(chat_history)
            input_ids = tokenizer.encode(input_text + tokenizer.eos_token, return_tensors="pt")

            # Generate response with the pre-trained model
            output_ids = model.generate(
                input_ids,
                max_length=200,
                pad_token_id=tokenizer.pad_token_id,
                temperature=0.7,
                top_k=40,
                top_p=0.8,
                do_sample=True,
                num_return_sequences=1,
                no_repeat_ngram_size=3
            )

            # Decode and print the response
            response = tokenizer.decode(output_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True).strip()
            response = adjust_tone(response)  # Adjust response tone
            chat_history.append(f"Chatbot: {response}")
            print(f"Chatbot: {response}")

        except KeyboardInterrupt:
            print("\nChatbot: Goodbye!")
            break

        except Exception as e:
            print(f"An error occurred: {e}")
            break

# Provide the path to your CSV file here
csv_file_path = "train.csv"  # Replace with your CSV file's path
chat_with_bot(csv_file_path)


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Chatbot is ready! Type 'exit' to end the chat.
You: hi
Chatbot (from CSV): I'm really sorry you're feeling this way. It's okay to feel down sometimes. If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media.  Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is somehow terrible.Bad feelings are part of living.  They are the motivation to remove ourselves from situations and relationships which do us more harm than good.Bad feelings do feel terrible.   Your feeling of worthle

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("fine_tuned_dialogpt")
model = AutoModelForCausalLM.from_pretrained("fine_tuned_dialogpt")

# Ensure the model is on the correct device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def minimal_test(prompt):
    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=500)
    input_ids = inputs["input_ids"].to(device)

    # Generate output with controlled parameters for focused and coherent responses
    outputs = model.generate(
        input_ids=input_ids,
        max_length=1000,         # Limit the response length to avoid unnecessary expansion
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,        # Enable sampling with tight constraints
        temperature=0.3,       # Keep the temperature low for determinism
        top_k=30,              # Limit the sampling pool to the top 30 tokens
        top_p=0.8,             # Top 80% cumulative probability to focus on more likely tokens
        repetition_penalty=3.0, # Higher penalty for repetition
        num_return_sequences=1,  # Return only one output
        no_repeat_ngram_size=3  # Avoid repeating 3-grams to ensure fluency
    )

    # Decode the output to text
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test with the prompt
print(minimal_test("Hi, how are you?"))


Hi, how are you? your. is and of I the therapist be to a


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import pandas as pd
from google.colab import files

# Load the pre-trained DialoGPT model and tokenizer
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")

# Add a padding token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

# Load CSV data
def load_csv_data(file_path):
    try:
        data = pd.read_csv(file_path)
        return data
    except Exception as e:
        print(f"Error loading CSV: {e}")
        return None

# Retrieve an answer from the CSV data based on a user query
def get_csv_response(data, query):
    if data is not None and "Context" in data.columns and "Response" in data.columns:
        for _, row in data.iterrows():
            # Lowercase for case-insensitive matching
            if query.lower() in row["Context"].lower():
                return row["Response"]
    return None

# Chatbot function with CSV and GPT integration
def chat_with_bot():
    print("Please upload your 'data.csv' file.")
    uploaded = files.upload()  # Allow file upload in Colab

    # Ensure the uploaded file is loaded
    if "data.csv" not in uploaded:
        print("No 'data.csv' file uploaded. Please try again.")
        return

    # Load the CSV data
    csv_data = load_csv_data("data.csv")
    if csv_data is None:
        print("Failed to load CSV. Exiting chatbot.")
        return

    print("\nChatbot is ready! Type 'exit' to end the chat.")
    chat_history_ids = None

    while True:
        try:
            user_input = input("You: ").strip()
            if user_input.lower() == 'exit':
                print("Goodbye!")
                break

            # Check if the CSV contains a related response
            csv_response = get_csv_response(csv_data, user_input)
            if csv_response:
                print(f"Chatbot (from CSV): {csv_response}")
                continue

            # Encode the new user input
            new_user_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

            # Prepare the bot's input (previous response + new user input)
            bot_input_ids = new_user_input_ids
            if chat_history_ids is not None:
                bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1)

            # Generate response
            chat_history_ids = model.generate(
                bot_input_ids,
                max_length=150,
                pad_token_id=tokenizer.pad_token_id,
                temperature=0.6,
                top_k=30,
                top_p=0.8,
                do_sample=True,
                num_return_sequences=1,
                no_repeat_ngram_size=3
            )

            # Decode the response
            response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

            # Print the response
            print(f"Chatbot: {response}")

        except KeyboardInterrupt:
            print("\nChatbot: Goodbye!")
            break
        except Exception as e:
            print(f"An error occurred: {e}")
            break

# Run the chatbot
chat_with_bot()


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-3.3-70B-Instruct.
401 Client Error. (Request ID: Root=1-677a6f4e-27e85c84693e47241781b033;751400e8-9475-4884-9f77-630ba7a7f67a)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.3-70B-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.3-70B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in.

In [ ]:
import streamlit as st
from pathlib import Path
from streamlit_chat import message
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
import os

os.environ["OPENAI_API_KEY"] = st.secrets["open_ai_api_key"]

st.title('CSV Question and answer ChatBot')


csv_file_uploaded = st.file_uploader(label="Upload your CSV File here")


if csv_file_uploaded is not None:
    def save_file_to_folder(uploadedFile):
        # Save uploaded file to 'content' folder.
        save_folder = 'content'
        save_path = Path(save_folder, uploadedFile.name)
        with open(save_path, mode='wb') as w:
            w.write(uploadedFile.getvalue())

        if save_path.exists():
            st.success(f'File {uploadedFile.name} is successfully saved!')

    save_file_to_folder(csv_file_uploaded)

    loader = CSVLoader(file_path=os.path.join('content/', csv_file_uploaded.name))

    # Create an index using the loaded documents
    index_creator = VectorstoreIndexCreator()
    docsearch = index_creator.from_loaders([loader])

    # Create a question-answering chain using the index
    chain = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=docsearch.vectorstore.as_retriever(), input_key="question")





    #Creating the chatbot interface
    st.title("Chat wtih your CSV Data")

        # Storing the chat
    if 'generated' not in st.session_state:
        st.session_state['generated'] = []

    if 'past' not in st.session_state:
        st.session_state['past'] = []


    def generate_response(user_query):
        response = chain({"question": user_query})
        return response['result']


    # We will get the user's input by calling the get_text function
    def get_text():
        input_text = st.text_input("You: ","Ask Question From your Document?", key="input")
        return input_text
    user_input = get_text()

    if user_input:
        output = generate_response(user_input)
        # store the output
        st.session_state.past.append(user_input)
        st.session_state.generated.append(output)

    if st.session_state['generated']:
        for i in range(len(st.session_state['generated'])-1, -1, -1):
            message(st.session_state["generated"][i], key=str(i))
            message(st.session_state['past'][i], is_user=True, key=str(i) + '_user')

ModuleNotFoundError: No module named 'streamlit'

In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.6 MB/s eta 0:00:00
